# Cogs 108 Final Project

Bernard Suwirjo

Ben Hazarian

Cesar Reyes

Eric Jiang

Gauthier Pred'Astor Gris

In [105]:
# Necessary Imports
import numpy as np
import pandas as pd

In [106]:
# Load Datasets

# Baseball Dataset
NFL_frame = pd.read_csv('data/NFL_16_17.csv')
NFL_frame.head()

,Unnamed: 0,Week,Day,Date,Time,Winner/tie,Unnamed: 5,Loser/tie,Unnamed: 7,Pts,Pts.1,YdsW,TOW,YdsL,TOL,Year
0,0,1,Thu,September 8,8:40PM,Denver Broncos,NaN,Carolina Panthers,boxscore,21.0,20.0,307.0,3.0,333.0,1.0,2016
1,1,1,Sun,September 11,1:04PM,Green Bay Packers,@,Jacksonville Jaguars,boxscore,27.0,23.0,294.0,0.0,348.0,1.0,2016
2,2,1,Sun,September 11,1:04PM,Baltimore Ravens,NaN,Buffalo Bills,boxscore,13.0,7.0,308.0,1.0,160.0,0.0,2016
3,3,1,Sun,September 11,1:05PM,Kansas City Chiefs,NaN,San Diego Chargers,boxscore,33.0,27.0,413.0,1.0,388.0,0.0,2016
4,4,1,Sun,September 11,1:05PM,Tampa Bay Buccaneers,@,Atlanta Falcons,boxscore,31.0,24.0,371.0,1.0,374.0,0.0,2016


Data cleaning

In [107]:
NFL_frame = NFL_frame.drop(columns=['Day','Time','Date', 'Unnamed: 7'])
NFL_frame.rename(columns={'Unnamed: 5':'Home_play'}, inplace=True)

Putting 1 if winning team played at home and 0 otherwise

In [108]:
for index, row in NFL_frame.iterrows():
    if row['Home_play'] =='@':
        NFL_frame.set_value(index, 'Home_play', 1)
    else:
        NFL_frame.set_value(index, 'Home_play', 0)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [109]:
NFL_frame.head()

,Unnamed: 0,Week,Winner/tie,Home_play,Loser/tie,Pts,Pts.1,YdsW,TOW,YdsL,TOL,Year
0,0,1,Denver Broncos,0,Carolina Panthers,21.0,20.0,307.0,3.0,333.0,1.0,2016
1,1,1,Green Bay Packers,1,Jacksonville Jaguars,27.0,23.0,294.0,0.0,348.0,1.0,2016
2,2,1,Baltimore Ravens,0,Buffalo Bills,13.0,7.0,308.0,1.0,160.0,0.0,2016
3,3,1,Kansas City Chiefs,0,San Diego Chargers,33.0,27.0,413.0,1.0,388.0,0.0,2016
4,4,1,Tampa Bay Buccaneers,1,Atlanta Falcons,31.0,24.0,371.0,1.0,374.0,0.0,2016
